# Pipeline processing using Dask

This notebook demonstrates the continuum imaging and ICAL pipelines.

In [1]:
%matplotlib inline

import os
import sys
sys.path.append(os.path.join('..','..'))

results_dir = './results'
os.makedirs(results_dir, exist_ok=True)

from dask import delayed
import dask
import dask.distributed

results_dir = './results'
os.makedirs(results_dir, exist_ok=True)

from matplotlib import pylab

pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy


import pprint

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs.utils import pixel_to_skycoord

from matplotlib import pyplot as plt

from arl.calibration.solvers import solve_gaintable
from arl.calibration.operations import apply_gaintable
from arl.calibration.calibration_control import create_calibration_controls
from arl.data.data_models import Image
from arl.data.polarisation import PolarisationFrame
from arl.data.parameters import get_parameter
from arl.visibility.base import create_blockvisibility
from arl.skycomponent.operations import create_skycomponent
from arl.image.deconvolution import deconvolve_cube
from arl.image.operations import show_image, export_image_to_fits, qa_image, copy_image, create_empty_image_like
from arl.visibility.iterators import vis_timeslice_iter
from arl.util.testing_support import create_named_configuration, create_low_test_beam
from arl.imaging import predict_2d, create_image_from_visibility, advise_wide_field

from arl.graphs.dask_init import get_dask_Client
from arl.graphs.delayed import create_invert_graph, create_predict_graph, create_deconvolve_facet_graph, \
    create_residual_graph, compute_list
from arl.imaging.imaging_context import imaging_context
from arl.graphs.generic_graphs import create_generic_image_graph
from arl.util.delayed_support import create_simulate_vis_graph, \
    create_low_test_image_from_gleam, create_corrupt_vis_graph
from arl.pipelines.delayed import create_continuum_imaging_pipeline_graph, \
    create_ical_pipeline_graph    

import logging

log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

In [2]:
from arl import arl_default_args
from arl.data.parameters import set_parameters, get_parameter

arl_default_args('imaging-pipelines.ini')

In [3]:
pp = pprint.PrettyPrinter()

Make a Dask client to manage the processing. Diagnostics are available at the URL given. Try the Status entry.

In [4]:
c=get_dask_Client()

Creating LocalCluster and Dask Client
<Client: scheduler='tcp://127.0.0.1:60897' processes=8 cores=8>
Diagnostic pages available on port http://127.0.0.1:8787


We create a graph to make the visibility. The parameter rmax determines the distance of the furthest antenna/stations used. All over parameters are determined from this number.

In [5]:
nfreqwin=7
ntimes=11
rmax=300.0
frequency=numpy.linspace(0.8e8,1.2e8,nfreqwin)
channel_bandwidth=numpy.array(nfreqwin*[frequency[1]-frequency[0]])
times = numpy.linspace(-numpy.pi/3.0, numpy.pi/3.0, ntimes)
phasecentre=SkyCoord(ra=+30.0 * u.deg, dec=-60.0 * u.deg, frame='icrs', equinox='J2000')

vis_graph_list=create_simulate_vis_graph('LOWBD2',
                                         frequency=frequency, 
                                         channel_bandwidth=channel_bandwidth,
                                         times=times,
                                         phasecentre=phasecentre,
                                         order='frequency',
                                        rmax=rmax)
print('%d elements in vis_graph_list' % len(vis_graph_list))
vis_graph_list = compute_list(c, vis_graph_list)

create_configuration_from_file: Maximum radius 300.0 m includes 94 antennas/stations
create_simulate_vis_graph: Simulating distribution in frequency
7 elements in vis_graph_list


In [6]:
wprojection_planes=1
advice_low=advise_wide_field(vis_graph_list[0], guard_band_image=4.0, delA=0.02,
                             wprojection_planes=wprojection_planes)

advice_high=advise_wide_field(vis_graph_list[-1], guard_band_image=4.0, delA=0.02,
                              wprojection_planes=wprojection_planes)

vis_slices = advice_low['vis_slices']
npixel=advice_high['npixels2']
cellsize=min(advice_low['cellsize'], advice_high['cellsize'])

convert_visibility: Original 11 rows, 0.004 GB, converted 48081 rows, 0.005 GB
advise_wide_field: Maximum wavelength 3.747 (meters)
advise_wide_field: Minimum wavelength 3.747 (meters)
advise_wide_field: Maximum baseline 152.4 (wavelengths)
advise_wide_field: Station/antenna diameter 35.0 (meters)
advise_wide_field: Primary beam 0.107069 (rad) 6.135 (deg)
advise_wide_field: Image field of view 0.428275 (rad) 24.538 (deg)
advise_wide_field: Synthesized beam 0.006560 (rad) 0.376 (deg)
advise_wide_field: Cellsize 0.002187 (rad) 0.125 (deg)
advice_wide_field: Npixels per side = 196
advice_wide_field: Npixels (power of 2, 3) per side = 256
advice_wide_field: W sampling for full image = 0.3 (wavelengths)
advice_wide_field: W sampling for primary beam = 5.6 (wavelengths)
advice_wide_field: Time sampling for full image = 62.6 (s)
advice_wide_field: Time sampling for primary beam = 1002.0 (s)
advice_wide_field: Frequency sampling for full image = 57983.7 (Hz)
advice_wide_field: Frequency sampli

Now make a graph to fill with a model drawn from GLEAM 

In [7]:
gleam_model_graph = [delayed(create_low_test_image_from_gleam)(npixel=npixel,
                                                               frequency=[frequency[f]],
                                                               channel_bandwidth=[channel_bandwidth[f]],
                                                               cellsize=cellsize,
                                                               phasecentre=phasecentre,
                                                               polarisation_frame=PolarisationFrame("stokesI"),
                                                               flux_limit=0.1,
                                                               applybeam=True)
                     for f, freq in enumerate(frequency)]

In [8]:
set_parameters('imaging-pipelines.ini', {'vis_slices':51, 'phase_error':1.0})
predicted_vis_graph_list = create_predict_graph(vis_graph_list, gleam_model_graph, context='wstack',
                                               arl_config='imaging-pipelines.ini')
predicted_vis_graph_list = compute_list(c, predicted_vis_graph_list)
corrupted_vis_graph_list = create_corrupt_vis_graph(predicted_vis_graph_list, arl_config='imaging-pipelines.ini')
corrupted_vis_graph_list = compute_list(c, corrupted_vis_graph_list)

/Users/timcornwell/anaconda/envs/arlenv/lib/python3.6/site-packages/distributed/worker.py:739: UserWarning: Large object of size 4.68 MB detected in task graph: 
  (<arl.data.data_models.BlockVisibility object at 0x11863b358>,)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s))


TypeError: predict_wstack_single() got an unexpected keyword argument 'context'

Get the LSM. This is currently blank.

In [ ]:
model_graph = [delayed(create_image_from_visibility)(vis_graph_list[f],
                                                     npixel=npixel,
                                                     frequency=[frequency[f]],
                                                     channel_bandwidth=[channel_bandwidth[f]],
                                                     cellsize=cellsize,
                                                     phasecentre=phasecentre,
                                                     polarisation_frame=PolarisationFrame("stokesI"))
               for f, freq in enumerate(frequency)]

Create a graph to make the dirty image 

In [ ]:
dirty_graph = create_invert_graph(predicted_vis_graph_list, model_graph, context='wstack',
                                  arl_config='imaging-pipelines.ini')
dirty_graph[0].visualize()

In [ ]:
from arl.graphs.dask_init import get_dask_Client
client = get_dask_Client()
dirty, sumwt = client.compute(dirty_graph[0], sync=True)
print(dirty)
show_image(dirty)
plt.show()

In [ ]:
set_parameters('imaging-pipelines.ini', {'facets':1, 'vis_slices':vis_slices}, 'imaging')

set_parameters('imaging-pipelines.ini', {'algorithm':'hogbom', 'niter':1000, 'fractional_threshold':0.1,
                                         'threshold':0.1, 'nmajor':5, 'gain':0.1}, 'deconvolution')

continuum_imaging_graph = \
    create_continuum_imaging_pipeline_graph(predicted_vis_graph_list, 
                                            model_graph=model_graph, 
                                            c_deconvolve_graph=create_deconvolve_facet_graph,
                                            arl_config='imaging-pipelines.ini'
                                            context='wstack', facets=1, vis_slices=vis_slices, 
                                            algorithm='hogbom', niter=1000, 
                                            fractional_threshold=0.1,
                                            threshold=0.1, nmajor=5, gain=0.1)

In [ ]:
future=c.compute(continuum_imaging_graph)

In [ ]:
deconvolved = future.result()[0][0]
residual = future.result()[1][0]
restored = future.result()[2][0]

f=show_image(deconvolved, title='Clean image - no selfcal')
print(qa_image(deconvolved, context='Clean image - no selfcal'))

plt.show()

f=show_image(restored, title='Restored clean image - no selfcal')
print(qa_image(restored, context='Restored clean image - no selfcal'))
plt.show()
export_image_to_fits(restored, '%s/imaging-dask_continuum_imaging_restored.fits' 
                     %(results_dir))

f=show_image(residual[0], title='Residual clean image - no selfcal')
print(qa_image(residual[0], context='Residual clean image - no selfcal'))
plt.show()
export_image_to_fits(residual[0], '%s/imaging-dask_continuum_imaging_residual.fits' 
                     %(results_dir))

In [ ]:
controls = create_calibration_controls()
        
controls['T']['first_selfcal'] = 2
controls['G']['first_selfcal'] = 3
controls['B']['first_selfcal'] = 4

controls['T']['timescale'] = 'auto'
controls['G']['timescale'] = 'auto'
controls['B']['timescale'] = 1e5

pp.pprint(controls)

In [ ]:
ical_graph = create_ical_pipeline_graph(corrupted_vis_graph_list, 
                                        model_graph=model_graph,  
                                        vis_slices=vis_slices,
                                        context='wstack',
                                        timeslice='auto',
                                        algorithm='hogbom', niter=1000, 
                                        fractional_threshold=0.1,
                                        threshold=0.1, nmajor=5, 
                                        gain=0.1, controls=controls,
                                        global_solution=False)

In [ ]:
future=c.compute(ical_graph)
deconvolved = future.result()[0][0]
residual = future.result()[1][0]
restored = future.result()[2][0]


f=show_image(deconvolved, title='Clean image')
print(qa_image(deconvolved, context='Clean image'))
plt.show()

f=show_image(restored, title='Restored clean image')
print(qa_image(restored, context='Restored clean image'))
plt.show()
export_image_to_fits(restored, '%s/imaging-dask_ical_restored.fits' 
                     %(results_dir))



f=show_image(residual[0], title='Residual clean image')
print(qa_image(residual[0], context='Residual clean image'))
plt.show()
export_image_to_fits(residual[0], '%s/imaging-dask_ical_residual.fits' 
                     %(results_dir))

In [ ]:
c.shutdown()